In [139]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({
        "question": question
    })
    memory.save_context({"input": question}, {"output": result.content},)
    print(result)


In [136]:
invoke_chain("My name is nico")

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "My name is nico"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel] Entering Chain run with input:
{
  "question": "My name is nico"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:load_memory] Entering Chain run with input:
{
  "question": "My name is nico"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:load_memory] s] Exiting Chain run with output:
{
  "output": []
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel] s] Exiting Chain run with output:
{
  "chat_history": []
}
[chain/start] [1:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "question": "My name is nico",
  "chat_history": []
}
[chain/end] [1:chain:RunnableSequence > 4:prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
{
  "lc": 1,
  "type": "constructor",
  "id": [
    "langchain",
    

In [137]:
invoke_chain("What is my name?")

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "question": "What is my name?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel] Entering Chain run with input:
{
  "question": "What is my name?"
}
[chain/start] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:load_memory] Entering Chain run with input:
{
  "question": "What is my name?"
}
[chain/end] [1:chain:RunnableSequence > 2:chain:RunnableParallel > 3:chain:load_memory] s] Exiting Chain run with output:
{
  "output": [
    {
      "lc": 1,
      "type": "constructor",
      "id": [
        "langchain",
        "schema",
        "messages",
        "HumanMessage"
      ],
      "kwargs": {
        "content": "My name is nico"
      }
    },
    {
      "lc": 1,
      "type": "constructor",
      "id": [
        "langchain",
        "schema",
        "messages",
        "AIMessage"
      ],
      "kwargs": {
        "content": "Hello Nico! How can I assist you today?"
   